In [0]:
%fs ls dbfs:/user/hive/warehouse/authentication_credentials

path,name,size,modificationTime
dbfs:/user/hive/warehouse/authentication_credentials/5u/,5u/,0,1723645016003
dbfs:/user/hive/warehouse/authentication_credentials/_delta_log/,_delta_log/,0,1723645016003


In [0]:
from pyspark.sql.types import * 
from pyspark.sql.functions import * # pyspark functions
# import urllib # URL processing



The following cell runs the notebooks which: 
- mounts the S3 bucket;
- stores utility functions.

In [0]:
%run ./mount_s3
%run ./functions

In [0]:
display(dbutils.fs.ls("/mnt/126ca3664fbb-mount/topics/126ca3664fbb.geo/"))
# to unmount
# dbutils.fs.unmount("/mnt/<mount name>")


path,name,size,modificationTime
dbfs:/mnt/126ca3664fbb-mount/topics/126ca3664fbb.geo/partition=0/,partition=0/,0,1723645017518


In [0]:
%sql
--Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


Reading in S3 JSON data as DataFrames

In [0]:
df_pin, df_geo, df_pin = [read_from_s3(i) for i in ['pin', 'geo', 'user']]

Cleaning/transforming the `pin` table.

In [0]:

df_pin = clean_pin_table(df_pin)
df_pin

Out[176]: DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, downloaded: bigint]

In [0]:
df_geo = clean_geo_table(df_geo)
df_geo

Out[177]: DataFrame[ind: bigint, country: string, coordinates: array<double>, timestamp: timestamp]

In [0]:
df_user = clean_user_table(df_user)
df_user

Out[178]: DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp]

In [0]:
# for reference

print(df_pin.columns)
print('--------------------------------------------')
print(df_geo.columns)
print('--------------------------------------------')
print(df_user.columns)

['ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category', 'downloaded']
--------------------------------------------
['ind', 'country', 'coordinates', 'timestamp']
--------------------------------------------
['ind', 'user_name', 'age', 'date_joined']

In [0]:
# to find the most popular category in each country
by_country = (
    df_pin.join(df_geo, 'ind', 'inner')
    .groupBy('country', 'category')
    .count()
    # .orderBy('count', ascending=False)
    .groupBy('country')
    .agg(
        first('category').alias('category'),
        max('count').alias('category_count')
        )
)

# to find the most popular category by year (2018-2022)
by_year = (
    df_pin.join(df_geo, 'ind', 'inner')
    .withColumn('post_year', year(col('timestamp')))
    .filter(col('post_year').between(2018, 2022))
    .groupBy('post_year', 'category')
    .count()
    # .withColumnRenamed('count', 'Total Count')
    .groupBy('post_year')
    .agg(
        first('category').alias('category'),
        max('count').alias('category_count')
    )
    .orderBy('post_year', ascending=True)
)

# to find the users with most followers per country
most_followers = (
    df_pin.join(df_geo, 'ind', 'inner')
    .groupBy('country','poster_name')
    .agg(max('follower_count').alias('follower_count'))
    .orderBy('country', ascending=True)
)

# find the country with the most followed user
country_most_followers = (
    most_followers.orderBy('follower_count', ascending=False)
    .select(col('country'),col('follower_count'))
    .limit(1)
)

# most popular category by age group
pop_age_group = (
    df_pin.join(df_user,'ind','inner')
    .withColumn("age_group",
    when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") > 24) & (col("age") < 36), "25-35")
    .when((col("age") > 35) & (col("age") < 50), "36-49")
    .otherwise("50+")
    )
    .groupBy('age_group', 'category')
    .count()
    .groupBy('age_group')
    .agg(
        first('category').alias('category'),
        max('count').alias('category_count'))
    .orderBy('age_group')
)

# median follower count by age group
fol_age_group = (
    df_pin.join(df_user,'ind','inner')
    .withColumn("age_group",
    when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") > 24) & (col("age") < 36), "25-35")
    .when((col("age") > 35) & (col("age") < 50), "36-49")
    .otherwise("50+")
    )
    .groupBy('age_group')
    .agg(
        expr("percentile_approx(follower_count, 0.5)")
        .alias('median_follower_count')
    )
    .orderBy('age_group')
)

# how many users joined in 2015-2020
join_year = (
    df_user.withColumn('year_joined',year(col("date_joined")))
    .filter(col('year_joined').between(2015, 2020))
    .groupBy('year_joined')
    .count()
    .withColumnRenamed('count', 'number_users_joined')
)

# median follower count by join year
fol_by_year = (
    df_user.join(
        df_pin, 'ind', 'inner'
    )
    .withColumn('year_joined',year(col("date_joined")))
    # .filter(col('year_joined').between(2015, 2020))
    .groupBy('year_joined')
    .agg(
        expr("percentile_approx(follower_count, 0.5)").alias('median_follower_count')
    )
)

# find the median follower count by age group and year joined
fol_age_year = (
    df_user.join(df_pin, 'ind', 'inner')
    .withColumn("age_group",
                when((col("age") >= 18) & (col("age") < 25), "18-24")
                .when((col("age") > 24) & (col("age") < 36), "25-35")
                .when((col("age") > 35) & (col("age") < 50), "36-49")
                .otherwise("50+")
    )
    .withColumn('year_joined',year(col("date_joined")))
    .groupBy('age_group', 'year_joined')
    .agg(
        expr("percentile_approx(follower_count, 0.5)").alias('median_follower_count')
    )
    .orderBy('age_group', 'year_joined')
)

# query outputs: uncomment to view results

# by_country.show()
# by_year.show()
# country_most_followers.show()
# most_followers.show()
# fol_age_group.show()
# fol_by_year.show()
fol_age_year.show()

+---------+-----------+---------------------+
age_group|year_joined|median_follower_count|
+---------+-----------+---------------------+
 18-24| 2015| 211000|
 18-24| 2016| 46000|
 18-24| 2017| 10000|
 25-35| 2015| 51000|
 25-35| 2016| 41000|
 25-35| 2017| 8000|
 36-49| 2015| 13000|
 36-49| 2016| 11000|
 36-49| 2017| 6000|
 50+| 2015| 196|
 50+| 2016| 1000|
 50+| 2017| 4000|
+---------+-----------+---------------------+